In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
race_results_df = spark.read.table("f1_presentation.race_results") \
    .filter(f"file_date = '{v_file_date}'")


In [0]:
from pyspark.sql.functions import sum, count, when, col, desc

In [0]:
race_year_list = df_column_to_list(race_results_df, "race_year")

In [0]:
driver_standings_df = race_results_df.groupBy("race_year", "driver_name","driver_nationality").agg(
    sum("points").alias("total_points"),
    count(when(col("position") == 1, True)).alias("wins")
) \
    .filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

final_df = driver_standings_df.withColumn("rank",rank().over(driver_rank_spec))


In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year"
merge_delta_data(final_df,'f1_presentation', 'driver_standings', merge_condition,'race_year')

In [0]:
%sql
select race_id,count(1) from f1_presentation.race_results
group by race_id
order by race_id desc
